In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pandas as pd
import torch

In [2]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.save_pretrained("tokenizer_roberta/token_izer")

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("./tokenizer_roberta/sentiment_tokenizer/")

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("./cardiffnlp/twitter-roberta-base-sentiment/")

In [28]:
def sentiment(row):
    text = row[:1500]
    encoded_input = tokenizer(text, return_tensors='pt')
    with torch.no_grad():  # Disable gradient calculation during inference
        output = model(**encoded_input)
    scores = output[0][0].cpu().numpy()  # Move scores to CPU for further processing
    scores = softmax(scores)


    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    max_score = 0
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        if s > max_score:
            final = l
            max_score = s
    return final

### News18

In [24]:
df = pd.read_excel(r"C:\Users\Dhruv\Downloads\sih\IndiaToday_Preprocessed.xlsx")

In [25]:
df = df.iloc[:100, :]

In [ ]:
df["Sentiment"] = df.Body.apply(lambda x: sentiment(str(x)))

In [8]:
df.head()

,Heading,Body,Category,URL,Sentiment
0,'Free speech should not be hate speech': Madra...,madras high court issue significant remark ami...,law,https://www.indiatoday.in/law/high-courts/stor...,neutral
1,"'Comments taken out of context', says US cop w...",seattle police officer guild friday come defen...,world,https://www.indiatoday.in/world/story/indian-s...,neutral
2,First meeting of 'One Nation One Election' com...,first official meeting one nation one election...,india,https://www.indiatoday.in/india/story/one-nati...,neutral
3,"US airlines’ flight depressurizes mid-air, plu...",united airlines jet head rome turn around less...,world,https://www.indiatoday.in/world/story/us-fligh...,neutral
4,3 terrorists killed as security forces foil in...,three terrorist kill infiltration bid foil sec...,india,https://www.indiatoday.in/india/story/one-terr...,negative


In [36]:
df.head()

,Heading,Body,Category,URL,Sentiment
0,'Free speech should not be hate speech': Madra...,madras high court issue significant remark ami...,law,https://www.indiatoday.in/law/high-courts/stor...,neutral
1,"'Comments taken out of context', says US cop w...",seattle police officer guild friday come defen...,world,https://www.indiatoday.in/world/story/indian-s...,neutral
2,First meeting of 'One Nation One Election' com...,first official meeting one nation one election...,india,https://www.indiatoday.in/india/story/one-nati...,neutral
3,"US airlines’ flight depressurizes mid-air, plu...",united airlines jet head rome turn around less...,world,https://www.indiatoday.in/world/story/us-fligh...,neutral
4,3 terrorists killed as security forces foil in...,three terrorist kill infiltration bid foil sec...,india,https://www.indiatoday.in/india/story/one-terr...,negative


In [80]:
file_name = "News18_with_sentiment.xlsx"
df.to_excel(file_name, index=False)

### IndiaToday 

In [85]:
df2 = pd.read_excel(r"C:\Users\Dhruv\Downloads\sih\IndiaToday_Preprocessed.xlsx")

In [92]:
df2["Sentiment"] = df2.Body.apply(lambda x: sentiment(str(x)))

In [93]:
file_name = "IndiaToday_with_sentiment.xlsx"
df2.to_excel(file_name, index=False)

### IndiaTV

In [88]:
df3 = pd.read_excel(r"C:\Users\Dhruv\Downloads\sih\IndiaTV_Preprocessed.xlsx")

In [90]:
df3["Sentiment"] = df3.Body.apply(lambda x: sentiment(x))

In [91]:
file_name = "IndiaTV_with_sentiment.xlsx"
df3.to_excel(file_name, index=False)